In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

%matplotlib inline

## Plot mask


- This mask was created by remapping `mask` variable in 1/20$^\circ$ OSTIA file to same lat-lon 1/8$^\circ$ regular grid that used in GEOS BCs
- Using `gen_daily_clim/save_mask.py`

### !ncdump -h data/mask_geos_bcs.nc

```
netcdf mask_geos_bcs {
dimensions:
	lon = 2880 ;
	lat = 1440 ;
variables:
    ...
    	float mask(lat, lon) ;
		mask:_FillValue = NaNf ;
		mask:long_name = "sea/land/lake/ice field composite mask" ;
		mask:flag_values = 1b, 2b, 4b, 8b ;
		mask:flag_meanings = "sea land lake ice" ;
		mask:comment = "b0: 1=grid cell is open sea water\nb1: 1=land is present in this grid cell\nb2: 1=lake surface is present in this grid cell\nb3: 1=sea ice is present in this grid cell\nb4-b7: reserved for future grid mask data" ;
```

## We got `mask`:
- = 1: open sea water
- = 2: land surface

## Build a new _custom_ land-sea mask using above information

In [ ]:
ls_mask = xr.open_dataset("data/mask_geos_bcs.nc")

my_mask = np.ones_like(ls_mask.mask.values)

# land
sVal=2.; Tol= 1.e-1
my_mask[( (ls_mask.mask.values> (sVal-Tol)) & (ls_mask.mask.values< (sVal+Tol)))] = 0.

# lake would be added like this; masking land suffices for now.
#sVal=4.; Tol= 1.e-1
#my_mask[( (ls_mask.mask.values> (sVal-Tol)) & (ls_mask.mask.values< (sVal+Tol)))] = 0.

### Save it

In [ ]:
da = xr.DataArray(data=my_mask, 
coords={'lat': ls_mask.lat,'lon': ls_mask.lon}, 
dims=["lat", "lon"],
attrs=dict(description="Land-sea mask (0: land, 1:otherwise) for GEOS forward processing applications"))

ds_lsMask = da.to_dataset(name='land_mask')

#ds_lsMask.land_mask.plot()
ds_lsMask.to_netcdf('data/geos_fp_bcs_land_sea_mask.nc')

### Plot it

In [ ]:
plt.figure( figsize=(8, 4))

plt.subplot(121)
ls_mask.mask.plot(vmin=0.5, vmax=14, cmap=plt.cm.Set1)
plt.title('From OSTIA')

plt.subplot(122)
ds_lsMask.land_mask.plot()
plt.title('Home made')